In [89]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [90]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [103]:
df_new = df.copy()
keys = [c for c in df_new if c.startswith('')]
pd.melt(df_new, value_name='transaction')
#set dataframe into one row 
# df_new = df.melt(value_name='transaction').drop('variable', 1)
# df_new

,variable,transaction
0,0,Bread
1,0,Bread
2,0,Cheese
3,0,Cheese
4,0,Meat
...,...,...
2200,6,NaN
2201,6,NaN
2202,6,Wine
2203,6,NaN


In [104]:
#df to list
df.agg(lambda x: list(x))
df

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,NaN,NaN,NaN,NaN
311,Meat,Milk,Pencil,NaN,NaN,NaN,NaN
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,NaN,NaN,NaN,NaN,NaN


In [106]:
#group df_new by transaction
transaction = df_new.groupby('transaction').count()
#create list from transaction
transaction_list = transaction.index.tolist()
#create set of transaction_list
product = set(transaction_list)
print(product)

KeyError: 'transaction'

## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [ ]:
#create an itemset based on the products
itemset = []
for i in product:
    itemset.append([i])
print(itemset)

# encoding the feature based on df
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(itemset).transform(itemset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

[['Bread'], ['Meat'], ['Diaper'], ['Bagel'], ['Wine'], ['Milk'], ['Eggs'], ['Pencil'], ['Cheese']]


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,False,True,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False
2,False,False,False,True,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True
5,False,False,False,False,False,False,True,False,False
6,False,False,False,False,True,False,False,False,False
7,False,False,False,False,False,False,False,True,False
8,False,False,True,False,False,False,False,False,False


In [ ]:

#df encoding
df["Bagel"] = df["Bagel"].astype(int)
df["Bread"] = df["Bread"].astype(int)
df["Cheese"] = df["Cheese"].astype(int)
df["Diaper"] = df["Diaper"].astype(int)
df["Eggs"] = df["Eggs"].astype(int)
df["Meat"] = df["Meat"].astype(int)
df["Milk"] = df["Milk"].astype(int)
df["Pencil"] = df["Pencil"].astype(int)
df["Wine"] = df["Wine"].astype(int)
df
# show the new dataframe



,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,0,1,0
8,0,0,1,0,0,0,0,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6